<a href="https://colab.research.google.com/github/nehmat-kaur/JSC270_A4_Group18/blob/main/JSC270_A4_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Part I: Sentiment Analysis with a Twitter Dataset**

In [ ]:
# imports (add whatever library you want to import)
import pandas as pd
import re
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# load the datasets (if you want to explore the datasets, do it here)
test_data = pd.read_csv('covid-tweets-test.csv')
train_data = pd.read_csv('covid-tweets-train.csv')

# I realized there are two columns in the train data
# with sentiment value 'PA"' and 'England"' so I'll remove them
train_data = train_data[(train_data['Sentiment'] != ' England\"') & (train_data['Sentiment'] != ' PA\"')]
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41153 entries, 0 to 41154
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     41153 non-null  int64 
 1   OriginalTweet  41153 non-null  object
 2   Sentiment      41151 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


A) Consider the training data. What is the balance between the three classes? In other words, what proportion of the observations (in the training set) belong to each class?


In [ ]:
# I wrote it in a function in case we need to do the same step in the future

def value_proportion(data, column: str):
  # count the occurrences of each sentiment
  # counts is an array of counts of each sentiment value
  counts = data[column].value_counts()

  # calculate the proportion of observations for each class
  total_samples = data.shape[0]
  proportions = counts / total_samples

  # return array of proportion for each sentiment value
  return proportions

proportions = value_proportion(train_data, "Sentiment")
for value, proportion in proportions.items():
    print(f"Sentiment {value}: {proportion:.2%}")


Sentiment 2: 43.84%
Sentiment 0: 37.41%
Sentiment 1: 18.74%


B) Tokenize the tweets. In other words, for each observation, convert the tweet from a single string of running text into a list of individual tokens (possibly with punctuation), splitting on whitespace. The result should be that each observation (tweet) is a list of individual tokens.


In [ ]:
nltk.download("punkt")

def tokenize(data):
  # create a new column in our DF that contains token lists instead of raw text
  data["tokens"] = data["OriginalTweet"].apply(nltk.word_tokenize)


tokenize(test_data)
print(test_data["tokens"].head(5))
tokenize(train_data)
print(train_data["tokens"].head(5))

test_data.info()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0    [TRENDING, :, New, Yorkers, encounter, empty, ...
1    [When, I, could, n't, find, hand, sanitizer, a...
2    [Find, out, how, you, can, protect, yourself, ...
3    [#, Panic, buying, hits, #, NewYork, City, as,...
4    [#, toiletpaper, #, dunnypaper, #, coronavirus...
Name: tokens, dtype: object
0    [@, MeNyrbie, @, Phil_Gahan, @, Chrisitv, http...
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia, :, Woolworths, to, gi...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me, ,, ready, to, go, at, supermarket, during...
Name: tokens, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     3798 non-null   int64 
 1   OriginalTweet  3798 non-null   object
 2   Sentiment      3798 non-null   int64 
 3   tokens         3798 non-null   object
dtypes: int64(2), object(2)
me

(C) Using a regular expression, remove any URL tokens from each of the observations.

In [ ]:
def remove_urls(tokens_df):
    # Define a regular expression pattern to match URLs
    url_pattern = re.compile(r'^https?')

    # Function to remove URLs from a single token list
    def clean_token_list(token_list):
        return [token for token in token_list if not re.match(url_pattern, token)]

    # Apply the function to each element in the DataFrame
    cleaned_tokens_df = tokens_df.applymap(lambda x: clean_token_list(x) if isinstance(x, list) else x)

    return cleaned_tokens_df

# Apply the function to the test data
test_data = remove_urls(test_data)
# Apply the function to the test data
train_data = remove_urls(train_data)


# Print the first few rows of the cleaned test data
print(test_data["tokens"].head(5))

# Print the first few rows of the cleaned train data
print(train_data["tokens"].head(5))

0    [TRENDING, :, New, Yorkers, encounter, empty, ...
1    [When, I, could, n't, find, hand, sanitizer, a...
2    [Find, out, how, you, can, protect, yourself, ...
3    [#, Panic, buying, hits, #, NewYork, City, as,...
4    [#, toiletpaper, #, dunnypaper, #, coronavirus...
Name: tokens, dtype: object
0    [@, MeNyrbie, @, Phil_Gahan, @, Chrisitv, :, /...
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia, :, Woolworths, to, gi...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me, ,, ready, to, go, at, supermarket, during...
Name: tokens, dtype: object


D) Remove all punctuation (,.?!;:’") and special characters(@, #, +, &, =, $, etc). Also, convert all tokens to lowercase only. Can you think of a scenario when you might want to keep some forms of punctuation?

In [ ]:
# Function to clean the tokens in a dataset
def clean_tokens(tokens_df):
    lowercase_tokens = []
    # create a list of lists with what we want
    for row in tokens_df["tokens"]:
      lowercase_tokens.append([t.lower() for t in row])

    # add the lower case tokens to our df
    tokens_df["lowercase_tokens"] = lowercase_tokens

    tokens_no_punct = []
    for row in tokens_df["lowercase_tokens"]:
      tokens_no_punct.append([re.sub('[^\w\s]','', t) for t in row])
    tokens_df["no_punct_tokens"] = tokens_no_punct

# Apply the function to the test data
clean_tokens(test_data)

# Apply the function to the test data
clean_tokens(train_data)

# Print the first few rows of the cleaned test data
print(test_data["no_punct_tokens"].head(5))

# Print the first few rows of the cleaned train data
print(train_data["no_punct_tokens"].head(5))

0    [trending, , new, yorkers, encounter, empty, s...
1    [when, i, could, nt, find, hand, sanitizer, at...
2    [find, out, how, you, can, protect, yourself, ...
3    [, panic, buying, hits, , newyork, city, as, a...
4    [, toiletpaper, , dunnypaper, , coronavirus, ,...
Name: no_punct_tokens, dtype: object
0    [, menyrbie, , phil_gahan, , chrisitv, , tcoif...
1    [advice, talk, to, your, neighbours, family, t...
2    [coronavirus, australia, , woolworths, to, giv...
3    [my, food, stock, is, not, the, only, one, whi...
4    [me, , ready, to, go, at, supermarket, during,...
Name: no_punct_tokens, dtype: object


E) Now stem your tokens. This will have the effect of converting similar word forms into identical tokens (e.g. run, runs, running → run). Please specify which stemmer you use.

In [ ]:
# Stemming tokens
stemmer = PorterStemmer()

# Function to stem tokens in a dataset
def stem_tokens(tokens_df):
    stemmed_tokens = []

    for row in tokens_df["no_punct_tokens"]:
      stemmed_tokens.append([stemmer.stem(w) for w in row])

    # add the lower case tokens to our df
    tokens_df["stemmed_tokens"] = stemmed_tokens

# Apply the function to the test data
stem_tokens(test_data)

# Apply the function to the test data
stem_tokens(train_data)

print(test_data["stemmed_tokens"].head(5))
print(train_data["stemmed_tokens"].head(5))

# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()

# # Function to lemmatize tokens in a dataset
# def lemmatize_tokens(tokens_df):
#     lemmatized_tokens = []

#     for row in tokens_df["no_punct_tokens"]:
#       lemmatized_tokens.append([lemmatizer.lemmatize(t) for t in row])

#     # add the lower case tokens to our df
#     tokens_df["lemmatized_tokens"] = lemmatized_tokens

# # Apply the function to the test data
# lemmatize_tokens(test_data)

# # Apply the function to the test data
# lemmatize_tokens(train_data)

# print(test_data["lemmatized_tokens"].head(5))
# print(train_data["lemmatized_tokens"].head(5))

0    [trend, , new, yorker, encount, empti, superma...
1    [when, i, could, nt, find, hand, sanit, at, fr...
2    [find, out, how, you, can, protect, yourself, ...
3    [, panic, buy, hit, , newyork, citi, as, anxio...
4    [, toiletpap, , dunnypap, , coronaviru, , coro...
Name: stemmed_tokens, dtype: object
0    [, menyrbi, , phil_gahan, , chrisitv, , tcoifz...
1    [advic, talk, to, your, neighbour, famili, to,...
2    [coronaviru, australia, , woolworth, to, give,...
3    [my, food, stock, is, not, the, onli, one, whi...
4    [me, , readi, to, go, at, supermarket, dure, t...
Name: stemmed_tokens, dtype: object


F) Lastly, remove stopwords. Using the english stopwords list from nltk, remove these common words from your observations. This list is very long (I think almost 200 words), so remove only the first 100 stopwords in the list.



In [ ]:
nltk.download('stopwords')
# print the top 75 most popular english words
sw = stopwords.words('english')[:100]


# Now let's remove them
def remove_stopwords(data):
  tokens_no_sw = []
  for row in data['stemmed_tokens']:
    tokens_no_sw.append([w for w in row if w not in sw])
  # Add column to df
  data['tokens_no_sw'] = tokens_no_sw

remove_stopwords(test_data)
remove_stopwords(train_data)
# Print some examples
print(train_data['stemmed_tokens'].head(5))
print(train_data['tokens_no_sw'].head(5))
print(test_data['tokens_no_sw'].head(5))

test_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0    [, menyrbi, , phil_gahan, , chrisitv, , tcoifz...
1    [advic, talk, to, your, neighbour, famili, to,...
2    [coronaviru, australia, , woolworth, to, give,...
3    [my, food, stock, is, not, the, onli, one, whi...
4    [me, , readi, to, go, at, supermarket, dure, t...
Name: stemmed_tokens, dtype: object
0    [, menyrbi, , phil_gahan, , chrisitv, , tcoifz...
1    [advic, talk, neighbour, famili, exchang, phon...
2    [coronaviru, australia, , woolworth, give, eld...
3    [food, stock, not, onli, one, empti, , pleas, ...
4    [, readi, go, supermarket, dure, , covid19, ou...
Name: tokens_no_sw, dtype: object
0    [trend, , new, yorker, encount, empti, superma...
1    [when, could, nt, find, hand, sanit, fred, mey...
2    [find, how, can, protect, love, one, , coronav...
3    [, panic, buy, hit, , newyork, citi, anxiou, s...
4    [, toiletpap, , dunnypap, , coronaviru, , coro...
Name: tokens_no_sw, dtype: object


,Unnamed: 0,OriginalTweet,Sentiment,tokens,lowercase_tokens,no_punct_tokens,stemmed_tokens,tokens_no_sw
0,0,TRENDING: New Yorkers encounter empty supermar...,0,"[TRENDING, :, New, Yorkers, encounter, empty, ...","[trending, :, new, yorkers, encounter, empty, ...","[trending, , new, yorkers, encounter, empty, s...","[trend, , new, yorker, encount, empti, superma...","[trend, , new, yorker, encount, empti, superma..."
1,1,When I couldn't find hand sanitizer at Fred Me...,2,"[When, I, could, n't, find, hand, sanitizer, a...","[when, i, could, n't, find, hand, sanitizer, a...","[when, i, could, nt, find, hand, sanitizer, at...","[when, i, could, nt, find, hand, sanit, at, fr...","[when, could, nt, find, hand, sanit, fred, mey..."
2,2,Find out how you can protect yourself and love...,2,"[Find, out, how, you, can, protect, yourself, ...","[find, out, how, you, can, protect, yourself, ...","[find, out, how, you, can, protect, yourself, ...","[find, out, how, you, can, protect, yourself, ...","[find, how, can, protect, love, one, , coronav..."
3,3,#Panic buying hits #NewYork City as anxious sh...,0,"[#, Panic, buying, hits, #, NewYork, City, as,...","[#, panic, buying, hits, #, newyork, city, as,...","[, panic, buying, hits, , newyork, city, as, a...","[, panic, buy, hit, , newyork, citi, as, anxio...","[, panic, buy, hit, , newyork, citi, anxiou, s..."
4,4,#toiletpaper #dunnypaper #coronavirus #coronav...,1,"[#, toiletpaper, #, dunnypaper, #, coronavirus...","[#, toiletpaper, #, dunnypaper, #, coronavirus...","[, toiletpaper, , dunnypaper, , coronavirus, ,...","[, toiletpap, , dunnypap, , coronaviru, , coro...","[, toiletpap, , dunnypap, , coronaviru, , coro..."
...,...,...,...,...,...,...,...,...
3793,3793,Meanwhile In A Supermarket in Israel -- People...,2,"[Meanwhile, In, A, Supermarket, in, Israel, --...","[meanwhile, in, a, supermarket, in, israel, --...","[meanwhile, in, a, supermarket, in, israel, , ...","[meanwhil, in, a, supermarket, in, israel, , p...","[meanwhil, supermarket, israel, , peopl, danc,..."
3794,3794,Did you panic buy a lot of non-perishable item...,0,"[Did, you, panic, buy, a, lot, of, non-perisha...","[did, you, panic, buy, a, lot, of, non-perisha...","[did, you, panic, buy, a, lot, of, nonperishab...","[did, you, panic, buy, a, lot, of, nonperish, ...","[panic, buy, lot, nonperish, item, , echo, nee..."
3795,3795,Asst Prof of Economics @cconces was on @NBCPhi...,1,"[Asst, Prof, of, Economics, @, cconces, was, o...","[asst, prof, of, economics, @, cconces, was, o...","[asst, prof, of, economics, , cconces, was, on...","[asst, prof, of, econom, , cconc, wa, on, , nb...","[asst, prof, econom, , cconc, wa, , nbcphilade..."
3796,3796,Gov need to do somethings instead of biar je r...,0,"[Gov, need, to, do, somethings, instead, of, b...","[gov, need, to, do, somethings, instead, of, b...","[gov, need, to, do, somethings, instead, of, b...","[gov, need, to, do, someth, instead, of, biar,...","[gov, need, someth, instead, biar, je, rakyat,..."


G) Now convert your lists of words into vectors of word counts. You may find Scikit-learn’s CountVectorizer useful here. What is the length of your vocabulary?

In [ ]:
X_test, y_test = test_data['tokens_no_sw'].to_numpy(), test_data['Sentiment'].to_numpy()
X_train, y_train = train_data['tokens_no_sw'].to_numpy(), train_data['Sentiment'].to_numpy()

def count_vectorize(data):
  # Assuming X contains tokenized tweets
  def override_fcn(token):
      # We expect a list of tokens as input, so we join them back into a string
      return token

  # Initialize CountVectorizer with the overridden tokenizer
  count_vec = CountVectorizer(
      analyzer='word',
      tokenizer=override_fcn,  # Use the overridden function
      preprocessor=override_fcn,  # Use the overridden function
      token_pattern=None,
  )

  counts = count_vec.fit_transform(X_train)
  return counts, count_vec

train_counts, count_vec = count_vectorize(train_data)
# transforms the testing data into a matrix of token counts (X_test_counts)
# using the same CountVectorizer instance count_vec as used for the training data.
test_counts = count_vec.transform(X_test)

# Count vector
print(test_counts.toarray())
print(train_counts.toarray())
# Print the length of the vocabulary
print("Length of vocabulary:", len(count_vec.vocabulary_))

[[11  0  0 ...  0  0  0]
 [11  0  0 ...  0  0  0]
 [ 3  0  0 ...  0  0  0]
 ...
 [ 8  0  0 ...  0  0  0]
 [ 6  0  0 ...  0  0  0]
 [ 6  0  0 ...  0  0  0]]
[[6 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]
 ...
 [7 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
Length of vocabulary: 74225


H) Recall the definition of the Naive Bayes model. If each document (tweet) is a collection of words (w1, · · · , wN ) belonging to class Ck (k = 0, 1, 2), then the Naive Bayes approach models the probability of each tweet belonging to class k:

The last equality follows from our “naive” assumption that words are conditionally independent given class. The probabilities are estimated using the frequencies of words within each class (bag of words), and we assign the class label according to which of the 3 posterior class probabilities (P(Ck|w1,··· ,wN)) is the highest.

Fit a Naive Bayes model to your data. Report the training and test error of the model. Use accuracy as the error metric. Also, report the 5 most probable words in each class, along with their counts. You might find Scikit-learn’s MultinomialNB() transformer useful. Use Laplace smoothing to prevent probabilities of zero.


In [ ]:
# y contains sentiment labels
# if test data has Nan values, we'll replace them with 1
for i in range(len(y_test)):
  if y_test[i] != 0 and y_test[i] != 1 and y_test[i] != 2:
    y_test[i] = 1

y_test = y_test.tolist()

# if training data has Nan values, we'll replace them with 1
for i in range(len(y_train)):
  if y_train[i] != '0' and y_train[i] != '1' and y_train[i] != '2':
    y_train[i] = 1

y_train = y_train.tolist()

# y_train has sentiment values in strings, so change them to integer
y_train = [int(label) for label in y_train]

# Train Naive Bayes model
nb = MultinomialNB()
nb.fit(train_counts, y_train)

# Predict on test data
y_test_preds = nb.predict(test_counts)
y_train_preds = nb.predict(train_counts)

# Print train and test errors
test_acc = accuracy_score(y_test, y_test_preds)
train_acc = accuracy_score(y_train, y_train_preds)

print('Test Accuracy: ', test_acc)
print('Train Accuracy: ', train_acc)

Test Accuracy:  0.6706161137440758
Train Accuracy:  0.820742108716254


In [ ]:
# getting top 5 words
print("Sentiment 0")
zero_count_list = nb.feature_count_[0].tolist()
zero_count_list_sorted = zero_count_list[:]
zero_count_list_sorted.sort()

top_5 = zero_count_list_sorted[-6:]
for count in top_5[0:5]:
  index = zero_count_list.index(count)
  print(f'word: {count_vec.get_feature_names_out()[index]}, count: {count}')

print("\nSentiment 1")
one_count_list = nb.feature_count_[1].tolist()
one_count_list_sorted = one_count_list[:]
one_count_list_sorted.sort()

top_5 = one_count_list_sorted[-6:]
for count in top_5[0:5]:
  index = one_count_list.index(count)
  print(f'word: {count_vec.get_feature_names_out()[index]}, count: {count}')

print("\nSentiment 2")
two_count_list = nb.feature_count_[2].tolist()
two_count_list_sorted = two_count_list[:]
two_count_list_sorted.sort()

top_5 = two_count_list_sorted[-6:]
for count in top_5[0:5]:
  index = two_count_list.index(count)
  print(f'word: {count_vec.get_feature_names_out()[index]}, count: {count}')

Sentiment 0
word: thi, count: 3225.0
word: food, count: 3638.0
word: price, count: 4347.0
word: covid19, count: 4610.0
word: coronaviru, count: 6737.0

Sentiment 1
word: price, count: 1365.0
word: supermarket, count: 1442.0
word: store, count: 1589.0
word: covid19, count: 2568.0
word: coronaviru, count: 3812.0

Sentiment 2
word: price, count: 3339.0
word: thi, count: 3781.0
word: store, count: 3917.0
word: covid19, count: 5681.0
word: coronaviru, count: 7511.0


J) (2 pts) Redo parts G-H using TF-IDF vectors instead of count vectors. You might find Scikitlearn’s TfidfVectorizer() transformer useful. Report the training and test accuracy. How does this compare to the accuracy using count vectors?  


In [ ]:
X_test, y_test = test_data['tokens_no_sw'].to_numpy(), test_data['Sentiment'].to_numpy()
X_train, y_train = train_data['tokens_no_sw'].to_numpy(), train_data['Sentiment'].to_numpy()

# G
def override_fcn(token):
    return token

def tfidf_vectorize(data):
    # Initialize TfidfVectorizer
    tfidf_vec = TfidfVectorizer(
        analyzer='word',
        tokenizer=override_fcn,  # Use a function that does nothing to the input tokens
        preprocessor=override_fcn,  # Use a function that does nothing to the input tokens
        token_pattern=None
    )
    # Fit and transform the tokenized tweets into TF-IDF vectors
    tfidf_matrix = tfidf_vec.fit_transform(data)
    return tfidf_matrix, tfidf_vec

# Apply TfidfVectorizer to training data
train_tfidf, tfidf_vec = tfidf_vectorize(X_train)
# Transform the testing data into TF-IDF vectors using the same TfidfVectorizer instance
test_tfidf = tfidf_vec.transform(X_test)

print(tfidf_vec.vocabulary_)
print(len(tfidf_vec.vocabulary_))

# y contains sentiment labels
# if test data has Nan values, we'll replace them with 1
for i in range(len(y_test)):
  if y_test[i] != 0 and y_test[i] != 1 and y_test[i] != 2:
    y_test[i] = 1

y_test = y_test.tolist()

# if training data has Nan values, we'll replace them with 1
for i in range(len(y_train)):
  if y_train[i] != '0' and y_train[i] != '1' and y_train[i] != '2':
    y_train[i] = 1

y_train = y_train.tolist()

# y_train has sentiment values in strings, so change them to integer
y_train = [int(label) for label in y_train]



# H
# Train Naive Bayes model
nb = MultinomialNB()
nb.fit(train_tfidf, y_train)


# Predict on test data
y_test_preds = nb.predict(test_tfidf)
y_train_preds = nb.predict(train_tfidf)

# Print train and test accuracy
test_acc = accuracy_score(y_test, y_test_preds)
train_acc = accuracy_score(y_train, y_train_preds)

print('Test Accuracy (TF-IDF): ', test_acc)
print('Train Accuracy (TF-IDF): ', train_acc)


# I dont know why the method i used for H doesnt work here, so i used new method
# getting top 5 words
log_probabilities = nb.feature_log_prob_

feature_names = tfidf_vec.get_feature_names_out()

# Get the count of each word in the training data
word_counts = train_tfidf.sum(axis=0)

# Convert to a flat array and retrieve the count for each feature (word)
word_counts = np.array(word_counts).flatten()

# Iterate over each sentiment category
for sentiment_label, log_probs in enumerate(log_probabilities):
    print(f"\nSentiment {sentiment_label}:")
    # Sort the words based on their log probabilities
    top_word_indices = np.argsort(log_probs)[::-1][1:6]
    # Print the top 5 words and their counts
    for index in top_word_indices:
        word = feature_names[index]
        count = word_counts[index]
        print(f"{word}: {count}")


{'': 0, 'menyrbi': 28530, 'phil_gahan': 34056, 'chrisitv': 9629, 'tcoifz9fan2pa': 53598, 'tcoxx6ghgfzcc': 65423, 'tcoi2nlzdxno8': 53359, 'advic': 3142, 'talk': 43185, 'neighbour': 30664, 'famili': 16722, 'exchang': 16392, 'phone': 34096, 'number': 31731, 'creat': 12119, 'contact': 10824, 'list': 26439, 'school': 38875, 'employ': 15748, 'chemist': 9369, 'gp': 19732, 'set': 39480, 'onlin': 32383, 'shop': 39894, 'account': 2899, 'poss': 34732, 'adequ': 3050, 'suppli': 42696, 'regular': 36729, 'med': 28306, 'not': 31506, 'order': 32582, 'coronaviru': 11221, 'australia': 5201, 'woolworth': 72353, 'give': 19202, 'elderli': 15523, 'disabl': 13999, 'dedic': 13279, 'hour': 21687, 'amid': 4049, 'covid19': 11688, 'outbreak': 32741, 'tcobinca9vp8p': 48338, 'food': 17728, 'stock': 42007, 'onli': 32382, 'one': 32316, 'empti': 15764, 'pleas': 34390, 'nt': 31698, 'panic': 33191, 'there': 67746, 'will': 72070, 'enough': 15881, 'everyon': 16320, 'take': 43138, 'more': 29504, 'than': 67423, 'need': 30627

K) (3 pts) Recall lemmatization converts each word to its base form, which is a bit stronger than simply taking the stem. Redo parts E-H using TF-IDF vectors instead of count vectors. This time use lemmatization instead of stemming. Report train and test accuracy. How does the accuracy with lemmatization compare to the accuracy with stemming?


In [ ]:
# E

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize tokens in a dataset
def lemmatize_tokens(tokens_df):
    lemmatized_tokens = []

    for row in tokens_df["no_punct_tokens"]:
        lemmatized_tokens.append([lemmatizer.lemmatize(t) for t in row])

    # Add the lemmatized tokens to our dataframe
    tokens_df["lemmatized_tokens"] = lemmatized_tokens

# Apply the function to the test data
lemmatize_tokens(test_data)

# Apply the function to the train data
lemmatize_tokens(train_data)

print(test_data["lemmatized_tokens"].head(5))
print(train_data["lemmatized_tokens"].head(5))


[nltk_data] Downloading package wordnet to /root/nltk_data...


0    [trending, , new, yorkers, encounter, empty, s...
1    [when, i, could, nt, find, hand, sanitizer, at...
2    [find, out, how, you, can, protect, yourself, ...
3    [, panic, buying, hit, , newyork, city, a, anx...
4    [, toiletpaper, , dunnypaper, , coronavirus, ,...
Name: lemmatized_tokens, dtype: object
0    [, menyrbie, , phil_gahan, , chrisitv, , tcoif...
1    [advice, talk, to, your, neighbour, family, to...
2    [coronavirus, australia, , woolworth, to, give...
3    [my, food, stock, is, not, the, only, one, whi...
4    [me, , ready, to, go, at, supermarket, during,...
Name: lemmatized_tokens, dtype: object


In [ ]:
# F

# Removing stopwords with lemmatization
def remove_stopwords(data):
    tokens_no_sw = []
    for row in data['lemmatized_tokens']:
        tokens_no_sw.append([w for w in row if w not in sw])
    # Add column to df
    data['tokens_no_sw'] = tokens_no_sw

remove_stopwords(test_data)
remove_stopwords(train_data)
# Print some examples
print(train_data['lemmatized_tokens'].head(5))
print(train_data['tokens_no_sw'].head(5))
print(test_data['tokens_no_sw'].head(5))


0    [, menyrbie, , phil_gahan, , chrisitv, , tcoif...
1    [advice, talk, to, your, neighbour, family, to...
2    [coronavirus, australia, , woolworth, to, give...
3    [my, food, stock, is, not, the, only, one, whi...
4    [me, , ready, to, go, at, supermarket, during,...
Name: lemmatized_tokens, dtype: object
0    [, menyrbie, , phil_gahan, , chrisitv, , tcoif...
1    [advice, talk, neighbour, family, exchange, ph...
2    [coronavirus, australia, , woolworth, give, el...
3    [food, stock, not, only, one, empty, , please,...
4    [, ready, go, supermarket, , covid19, outbreak...
Name: tokens_no_sw, dtype: object
0    [trending, , new, yorkers, encounter, empty, s...
1    [when, could, nt, find, hand, sanitizer, fred,...
2    [find, how, can, protect, loved, one, , corona...
3    [, panic, buying, hit, , newyork, city, anxiou...
4    [, toiletpaper, , dunnypaper, , coronavirus, ,...
Name: tokens_no_sw, dtype: object


In [ ]:
# G

from sklearn.feature_extraction.text import TfidfVectorizer

# Function to perform TF-IDF vectorization
def tfidf_vectorize(data):
    # Assuming X contains tokenized tweets and y contains sentiment labels
    X = data['tokens_no_sw'].apply(lambda x: ' '.join(x))

    # Initialize TfidfVectorizer
    tfidf_vec = TfidfVectorizer()

    # Fit and transform the tokenized tweets into TF-IDF vectors
    tfidf_matrix = tfidf_vec.fit_transform(X)
    return tfidf_matrix, tfidf_vec

test_tfidf_matrix, test_tfidf_vec = tfidf_vectorize(test_data)
train_tfidf_matrix, train_tfidf_vec = tfidf_vectorize(train_data)

print(test_tfidf_matrix.toarray())
# Print the vocabulary
print(test_tfidf_vec.vocabulary_)
# Print the length of the vocabulary
print("Length of vocabulary:", len(test_tfidf_vec.vocabulary_))

print(train_tfidf_matrix.toarray())
# Print the vocabulary
print(train_tfidf_vec.vocabulary_)
# Print the length of the vocabulary
print("Length of vocabulary:", len(train_tfidf_vec.vocabulary_))


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'trending': 12218, 'new': 6841, 'yorkers': 13131, 'encounter': 3587, 'empty': 3574, 'supermarket': 9825, 'shelf': 9054, 'pictured': 7607, 'wegmans': 12807, 'brooklyn': 1568, 'soldout': 9364, 'online': 7138, 'grocer': 4609, 'foodkick': 4160, 'maxdelivery': 6345, 'coronavirusfearing': 2496, 'shopper': 9113, 'stock': 9637, 'tcogr76pcrlwh': 10674, 'tcoivmkmsqdt1': 10785, 'when': 12865, 'could': 2577, 'nt': 7004, 'find': 4031, 'hand': 4713, 'sanitizer': 8777, 'fred': 4258, 'meyer': 6456, 'turned': 12309, 'amazon': 723, '11497': 46, 'pack': 7305, 'purell': 8053, 'check': 1956, 'how': 5009, 'coronavirus': 2479, 'concern': 2268, 'driving': 3341, 'price': 7890, 'tcoygbipbflmy': 11708, 'can': 1727, 'protect': 7987, 'loved': 6119, 'one': 7132, 'panic': 7354, 'buying': 1659, 'hit': 4902, 'newyork': 6860, 'city': 2044, 'anxious': 830, 'food': 4

In [ ]:
print("Shape of train TF-IDF matrix:", train_tfidf_matrix.shape)
print("Shape of test TF-IDF matrix:", test_tfidf_matrix.shape)


Shape of train TF-IDF matrix: (41153, 80944)
Shape of test TF-IDF matrix: (3798, 13326)


In [ ]:
# H

# Train Naive Bayes model with TF-IDF vectors
nb_tfidf = MultinomialNB()
nb_tfidf.fit(train_tfidf_matrix, y_train)

# Use the TF-IDF vectorizer trained on the training data to transform the test data
test_tfidf_matrix = train_tfidf_vec.transform(test_data['tokens_no_sw'].apply(lambda x: ' '.join(x)))

# Predict on test data
y_test_preds_tfidf = nb_tfidf.predict(test_tfidf_matrix)
y_train_preds_tfidf = nb_tfidf.predict(train_tfidf_matrix)

# Print train and test errors
test_acc_tfidf = accuracy_score(y_test, y_test_preds_tfidf)
train_acc_tfidf = accuracy_score(y_train, y_train_preds_tfidf)

print('Test Accuracy with lemmatization: ', test_acc_tfidf)
print('Train Accuracy with lemmatization: ', train_acc_tfidf)


Test Accuracy with TF-IDF vectors:  0.6274354923644023
Train Accuracy with TF-IDF vectors:  0.7354992345637014
